In [ ]:
import os
import cv2

import pandas as pd
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from shapely import wkt
from rasterio import features

In [ ]:
aoi_name = 'L15-0457E-1135N_1831_3648_13'
file_name = 'global_monthly_2019_12_mosaic_L15-0457E-1135N_1831_3648_13'

df_path = '../../../change_detection_dataset/dataset_1/all_df.csv'

aoi_path = Path('../../../datasets/space_net_7/SN7_buildings_train/train/{}/images_masked/'.format(aoi_name))
output_image = '../../../building_detection_material/before_and_after_filtering.png'

### Read CSV

In [ ]:
df = pd.read_csv(df_path, index_col=0)

df = df[(df['AOI_name'] == aoi_name) & (df['filename'] == file_name)]
df

In [ ]:
df['area'] = df['geometry'].apply(lambda x: wkt.loads(x).area)

In [ ]:
df['area'].describe()

In [ ]:
r = rio.open(aoi_path / '.'.join([file_name, 'tif'])).read()

image = r.transpose((1, 2, 0))[:, :, :-1]

polygons_small = list(df[df['area']<=50]['geometry'])
polygons_large = list(df[df['area']>50]['geometry'])

mask_large_buildings = features.rasterize(((wkt.loads(polygon), (1)) for polygon in polygons_large), out_shape=image.shape[:2])
mask_small_buildings = features.rasterize(((wkt.loads(polygon), (1)) for polygon in polygons_small), out_shape=image.shape[:2])

In [ ]:
mask = np.ones((image.shape[0], image.shape[1], 3) , dtype=np.uint8) * 255

In [ ]:
mask[mask_large_buildings == 1] = [36, 110, 12]
mask[mask_small_buildings == 1] = [150, 17, 17]

In [ ]:
# mask[:, :, 0][mask_small_buildings == 1] = 221
# mask[:, :, 1][mask_large_buildings == 1] = 221

In [ ]:
fig, ax = plt.subplots(figsize=(24, 24))

plt.imshow(image)
plt.imshow(mask, alpha=0.4)

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax.set_axis_off()


plt.savefig(output_image, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12), gridspec_kw = {'wspace':0, 'hspace':0})

ax[0].imshow(image)
ax[0].set_axis_off()

ax[1].imshow(mask)

ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)

ax[1].set_axis_off()

plt.savefig('_1.'.join(output_image.split('.')), bbox_inches='tight')